### Imports

In [ ]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from pprint import pprint

from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve

from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.inspection import permutation_importance

from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

Divisão do dataset em treino e em teste.

In [ ]:
X = dataset.drop('Y', axis=1)
y = dataset['Y']

train_X, train_y, test_X, test_y= train_test_split(X, y, test_size=0.2, random_state=42)

# Modelos de ML
A nossa task, prever o nivel de sinergia entre duas drogas, o que se trata de um problema de regressão, logo só poderemos utilizar modelos que se baseiem em regressão.

## Logistical Regression

Um modelo de regressão logística modela a relação entre as variáveis independentes e a probabilidade de pertencer a uma determinada classe usando uma função logística. 

Aqui os hiperparâmetros estão definidos, mas podiamos fazer algo para determinar a melhro combinação.

Depois de cada modelo já feito com parâmetreos definidos teoricamente, tem outro exemplo com a exploração dos melhores hiperparâmetros. Para discutir a melhor abordagem

In [ ]:
logreg = LogisticRegression(random_state=45, max_iter=1000)
logreg.fit(train_X, train_y)

print(f'Accuracy: {logreg.score(test_X, test_y):.2%}')

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

pred_y = logreg.predict(test_X)
pprint(classification_report(test_y, pred_y))

In [ ]:
print('Confusion Matrix:')
confusion_matrix(test_y, pred_y)

In [ ]:
#define metrics
y_pred_proba = logreg.predict_proba(test_y)[::,1]
fpr, tpr, _ = roc_curve(test_y,  y_pred_proba)
auc = roc_auc_score(test_y, y_pred_proba)

#create ROC curve
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

#### Com exploração de hiperparâmetros para determinar a melhor combinação

In [ ]:
# Definir os hiperparâmetros 
parameters = {
    'penalty': ['l1', 'l2', 'none'],
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs', 'saga'],
    'max_iter': [10,100, 200]
}

# Criar o modelo de regressão logística
logistic_regression = LogisticRegression()

# Criar o objeto GridSearchCV
grid_search = GridSearchCV(logistic_regression, parameters, scoring='f1', cv=5)

# Executar a busca em grid para encontrar os melhores hiperparâmetros
grid_search.fit(train_X[:, 3:], train_y)  #FUI USANDO SEMPRE [:, 3:] (todas as linhas e descartando as 3 primeiras colunas) MAS NÃO ESTOU A VER NO DATASET QUAIS COLUNAS INCLUIR, POR ISSO QUANDO SE TIVER O DATASET TEM QUE SE ADAPTAR ISTO EM TODOS ABAIXO

# Obter os melhores hiperparâmetros encontrados
best_params_LT = grid_search.best_params_
best_params_LT

In [ ]:
#TREINO
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit

# Supondo que você tenha um conjunto de dados train_X, train_y
# e que 'best_params_LT' contenha os melhores hiperparâmetros obtidos

# Criar um objeto MinMaxScaler
min_max_scaler = MinMaxScaler()

# Criar o modelo Logistic Regression com os melhores hiperparâmetros
LR_classifier = LogisticRegression(random_state=42, **best_params_LT)

# Dividir os dados usando StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

for train_index, test_index in sss.split(train_X, train_y):
    X_LR_train, X_LR_test, y_LR_train, y_LR_test =  train_X[train_index, 3:], train_X[test_index, 3:], train_y[train_index], train_y[test_index]
    
    # Normalizar os dados
    X_LR_train_new = min_max_scaler.fit_transform(X_LR_train)
    X_LR_test_new = min_max_scaler.transform(X_LR_test)
    
    # Treinar o modelo
    LR_classifier.fit(X_LR_train_new, y_LR_train)
    
    # Fazer previsões
    y_pred = LR_classifier.predict(X_LR_test_new)
    

In [ ]:
# TESTE
import pandas as pd
X_te_new = test_X[:,3:]
x_test_df = pd.DataFrame(X_te_new)

X_te_new = min_max_scaler.transform(X_te_new)
x_test_df = pd.DataFrame(X_te_new)

y_LR_pred = LR_classifier.predict(X_te_new)

print("*************************************")
print('f1 on Test set: ', f1_score(y_te, y_LR_pred))
print('MCC on Test set: ', matthews_corrcoef(y_te, y_LR_pred))
tn, fp, fn, tp = confusion_matrix(y_te, y_LR_pred).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn + fp)
print('Specificity on Test set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp + fn)
print('Sensitivity on Test set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp + tn) / (tp + tn + fp + fn)
print('Accuracy on Test set: ', accuracy)
precision=tp/(tp+fp)
print("Precision on Test set: ", precision)

In [ ]:
# Calcula a matriz de confusão
confusion = confusion_matrix(test_y, y_LR_pred)

# Obtém os valores dos verdadeiros positivos (tp), verdadeiros negativos (tn),
# falsos positivos (fp) e falsos negativos (fn)
tn, fp, fn, tp = confusion.ravel()

# Imprime os valores da matriz de confusão
print("Confusion Matrix:")
print(confusion)

# Plota a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, cmap='Greens', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

## Decision Tree

Árvore de decisão é um modelo que se assemelha a um fluxograma que toma decisões ou previsões, dividindo recursivamente os dados com base em diferentes carateristicas.

In [ ]:
dtree = DecisionTreeRegressor(random_state=45)
dtree.fit(train_X, train_y)

print(f'Accuracy: {dtree.score(test_X, test_y):.2%}')

pred_y = dtree.predict(test_X)
print(classification_report(test_y, pred_y))

#### Com exploração de hiperparâmetros para determinar a melhor combinação

In [ ]:
#hiperparametros
parameters = {
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

dt_classifier = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(dt_classifier, parameters, scoring='f1', cv=5)
grid_search.fit(X_tr[:, 3:], train_y)

best_params_DT = grid_search.best_params_
best_params_DT

In [ ]:
#TREINO
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.preprocessing import MinMaxScaler

# Supondo que você tenha um conjunto de dados train_X, train_y
# e que 'best_params_DT' contenha os melhores hiperparâmetros obtidos

# Criar um objeto MinMaxScaler
min_max_scaler = MinMaxScaler()

# Criar o modelo Decision Tree Classifier com os melhores hiperparâmetros
DT_classifier = DecisionTreeClassifier(random_state=42, **best_params_DT)

# Dividir os dados usando StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

for train_index, test_index in sss.split(train_X, train_y):
    X_DT_train, X_DT_test, y_DT_train, y_DT_test = train_X[train_index, 3:], train_X[test_index, 3:], train_y[train_index], train_y[test_index]
    
    # Normalizar os dados
    X_DT_train_new = min_max_scaler.fit_transform(X_DT_train)
    X_DT_test_new = min_max_scaler.transform(X_DT_test)
    
    # Treinar o modelo
    DT_classifier.fit(X_DT_train_new, y_DT_train)
    
    # Fazer previsões
    y_pred = DT_classifier.predict(X_DT_test_new)
    
    # Restante do seu código para avaliação do modelo, por exemplo, métricas de desempenho

   

In [ ]:
# Avaliação

import pandas as pd
#AVALIAÇÃO
X_te_new = X_te[:,3:]
x_test_df = pd.DataFrame(X_te_new)

X_te_new = min_max_scaler.transform(X_te_new)
x_test_df = pd.DataFrame(X_te_new)

y_DT_pred = DT_classifier.predict(X_te_new)

print("*************************************")
print('f1 on Test set: ', f1_score(y_te, y_DT_pred))
print('MCC on Test set: ', matthews_corrcoef(y_te, y_DT_pred))
tn, fp, fn, tp = confusion_matrix(y_te, y_DT_pred).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn + fp)
print('Specificity on Test set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp + fn)
print('Sensitivity on Test set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp + tn) / (tp + tn + fp + fn)
print('Accuracy on Test set: ', accuracy)
precision=tp/(tp+fp)
print("Precision on Test set: ", precision)


In [ ]:
# Confusion Matrix
# Calcula a matriz de confusão
confusion = confusion_matrix(y_te, y_DT_pred)

# Obtém os valores dos verdadeiros positivos (tp), verdadeiros negativos (tn),
# falsos positivos (fp) e falsos negativos (fn)
tn, fp, fn, tp = confusion.ravel()

# Imprime os valores da matriz de confusão
print("Confusion Matrix:")
print(confusion)

# Plota Da matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, cmap='Greens', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

## Random Forest

Random Forest é um método de aprendizagem que combina várias árvores de decisão para fazer previsões, e que suporta regressão e classificação. Este modelo pode não ser o mais adequado quando o dataset não é equilibrado.

In [ ]:
rfc = RandomForestRegressor(random_state=45)
rfc.fit(train_X, train_y)

print(f'Accuracy: {rfc.score(test_X, test_y):.2%}')

pred_y = rfc.predict(test_X)
pprint(classification_report(test_y, pred_y))

#### Com exploração de hiperparâmetros para determinar a melhor combinação

In [ ]:
# Definir os parâmetros a ajustar
parameters = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

# Criar o classificador RandomForest
RF_classifier = RandomForestClassifier(random_state=42)

# Criar o objeto GridSearchCV
grid_search = GridSearchCV(RF_classifier, parameters, scoring='f1', cv=5)

# Executar o grid search usando os dados de treino (X_tr e y_tr)
grid_search.fit(X_tr[:, 3:], y_tr)

# Obter os melhores parâmetros
best_params_RF = grid_search.best_params_
best_params_RF

In [ ]:
# TREINO
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.preprocessing import MinMaxScaler

# Supondo que você tenha um conjunto de dados train_X, train_y
# e que 'best_params_RF' contenha os melhores hiperparâmetros obtidos

# Criar um objeto MinMaxScaler
min_max_scaler = MinMaxScaler()

# Criar o modelo RandomForestClassifier com os melhores hiperparâmetros
RF_classifier = RandomForestClassifier(random_state=42, **best_params_RF)

# Dividir os dados usando StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

for train_index, test_index in sss.split(train_X, train_y):
    X_RF_train, X_RF_test, y_RF_train, y_RF_test = train_X[train_index, 3:], train_X[test_index, 3:], train_y[train_index], train_y[test_index]
    
    # Normalizar os dados
    X_RF_train_new = min_max_scaler.fit_transform(X_RF_train)
    X_RF_test_new = min_max_scaler.transform(X_RF_test)
    
    # Treinar o modelo
    RF_classifier.fit(X_RF_train_new, y_RF_train)
    
    # Fazer previsões
    y_pred = RF_classifier.predict(X_RF_test_new)



In [ ]:
# Avaliação
X_new = train_X[:,3:]
X_new = min_max_scaler.fit_transform(X_new)
RF_classifier.fit(X_new, train_y)
y_tr_predict = RF_classifier.predict(X_new)
print('f1 on Train set: ', f1_score(y_tr, y_tr_predict))
print('MCC on Train set: ', matthews_corrcoef(train_y, y_tr_predict))

tn, fp, fn, tp = confusion_matrix(y_tr, y_tr_predict).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn+fp)
print('Specificity on Train set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp+fn)
print('Sensitivity on Train set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp+tn) /(tp+tn+fp+fn)
print('Accuracy on Train set: ', accuracy)

In [ ]:
#Teste
X_te_new = X_te[:,3:]
x_test_df = pd.DataFrame(X_te_new)
X_te_new = min_max_scaler.transform(X_te_new)
x_test_df = pd.DataFrame(X_te_new)
y_RF_pred=RF_classifier.predict(X_te_new)
print("*************************************")
print('f1 on Test set: ', f1_score(y_te, y_RF_pred))
print('MCC on Test set: ', matthews_corrcoef(y_te, y_RF_pred))
tn, fp, fn, tp = confusion_matrix(y_te, y_RF_pred).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn+fp)
print('Specificity on Test set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp+fn)
print('Sensitivity on Test set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp+tn) /(tp+tn+fp+fn)
print('Accuracy on Test set: ', accuracy)
precision=tp/(tp+fp)
print("Precision on Test set: ", precision)

In [ ]:
#Confusion Matrix
# Calcula a matriz de confusão
confusion = confusion_matrix(test_t, y_RF_pred)

# Obtém os valores dos verdadeiros positivos (tp), verdadeiros negativos (tn),
# falsos positivos (fp) e falsos negativos (fn)
tn, fp, fn, tp = confusion.ravel()

# Imprime os valores da matriz de confusão
print("Confusion Matrix:")
print(confusion)

# Plot da matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, cmap='Greens', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

## SVR (Support Vector Regression)

O SVR é uma técnica de ML para regressão, baseada em Support Vector Machines (SVM), que procura prever valores contínuos ao otimizar uma função de margem de erro entre as previsões do modelo e os valores reais dos dados de treino

In [ ]:
svr = SVR()
svr.fit(train_X, train_y)

print(f'Accuracy: {svr.score(test_X, test_y):.2%}')

pred_y = svr.predict(test_X)
pprint(classification_report(test_y, pred_y))

#### Com exploração de hiperparâmetros para determinar a melhor combinação

In [ ]:
# Definir os parâmetros a ajustar para SVR
parameters_svr = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto', 0.1, 1],
    'epsilon': [0.1, 0.2, 0.5]
}

# Criar o regressor SVR
SVR_regressor = SVR()

# Criar o objeto GridSearchCV
grid_search_svr = GridSearchCV(SVR_regressor, parameters_svr, scoring='neg_mean_squared_error', cv=5)

# Executar o grid search usando os dados de treino (X_tr e y_tr)
grid_search.fit(train_X[:, 3:], train_y)

# Obter os melhores parâmetros
best_params_SVR = grid_search.best_params_
best_params_SVR


In [ ]:
#TREINO

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

# Supondo que você tenha um conjunto de dados train_X, train_y
# e que 'best_params_SVM' contenha os melhores hiperparâmetros obtidos

# Criar um objeto MinMaxScaler
min_max_scaler = MinMaxScaler()

# Definir os parâmetros a ajustar para SVM
parameters_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto', 0.1, 1],
    'degree': [2, 3, 4],
}

# Criar o classificador SVM
SVM_classifier = SVC()

# Criar o objeto GridSearchCV
grid_search_svm = GridSearchCV(SVM_classifier, parameters_svm, scoring='accuracy', cv=5)

# Dividir os dados usando StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

for train_index, test_index in sss.split(train_X, train_y):
    X_SVM_train, X_SVM_test, y_SVM_train, y_SVM_test = train_X[train_index, 3:], train_X[test_index, 3:], train_y[train_index], train_y[test_index]
    
    # Normalizar os dados
    X_SVM_train_new = min_max_scaler.fit_transform(X_SVM_train)
    X_SVM_test_new = min_max_scaler.transform(X_SVM_test)
    
    # Executar o grid search usando os dados de treino
    grid_search_svm.fit(X_SVM_train_new, y_SVM_train)

    # Obter os melhores parâmetros
    best_params_SVM = grid_search_svm.best_params_
    
    # Criar o modelo SVM com os melhores hiperparâmetros
    SVM_classifier = SVC(**best_params_SVM)
    
    # Treinar o modelo
    SVM_classifier.fit(X_SVM_train_new, y_SVM_train)


In [ ]:
#Avaliação
X_new = train_X[:,3:]
X_new = min_max_scaler.fit_transform(X_new)
KN_classifier.fit(X_new, y_tr)
y_tr_predict = KN_classifier.predict(X_new)
print('f1 on Train set: ', f1_score(y_tr, y_tr_predict))
print('MCC on Train set: ', matthews_corrcoef(y_tr, y_tr_predict))

tn, fp, fn, tp = confusion_matrix(y_tr, y_tr_predict).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn+fp)
print('Specificity on Train set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp+fn)
print('Sensitivity on Train set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp+tn) /(tp+tn+fp+fn)
print('Accuracy on Train set: ', accuracy)

In [ ]:
#TESTE

X_te_new = test_X[:,3:]

X_te_new = min_max_scaler.transform(X_te_new)

y_SVM_pred = KN_classifier.predict(X_te_new)

print("*************************************")
print('f1 on Test set: ', f1_score(y_te, y_SVM_pred))
print('MCC on Test set: ', matthews_corrcoef(y_te, y_SVM_pred))
tn, fp, fn, tp = confusion_matrix(y_te, y_SVM_pred).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn + fp)
print('Specificity on Test set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp + fn)
print('Sensitivity on Test set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp + tn) / (tp + tn + fp + fn)
print('Accuracy on Test set: ', accuracy)
precision=tp/(tp+fp)
print("Precision on Test set: ", precision)

In [ ]:
#Confusion Matrix

# Calcula a matriz de confusão
confusion = confusion_matrix(y_te, y_SVM_pred)

# Obtém os valores dos verdadeiros positivos (tp), verdadeiros negativos (tn),
# falsos positivos (fp) e falsos negativos (fn)
tn, fp, fn, tp = confusion.ravel()

# Imprime os valores da matriz de confusão
print("Confusion Matrix:")
print(confusion)

# Plota a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, cmap='Greens', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

IMPORTANTE: Podemos fazer como está em cima um de cada vez, ou entao, se quisermos treinar e testar todos os modelos de uma vez o prof usou isto na aula 10, ja pus os modelos que vi que deviamos usar, vejam se faz sentido.

In [ ]:
models = [LogisticRegression(random_state=42, max_iter=1000),
          DecisionTreeRegressor(random_state=42),
          RandomForestRegressor(random_state=42),
          SVR()]

for model in models:
    model.fit(train_X, train_y)
    print(model.__class__.__name__)
    print('Accuracy on test set:', model.score(test_X, test_y))
    print('Classification report:\n', classification_report(test_y, model.predict(test_X)))
    print('Confusion matrix:\n', confusion_matrix(test_y, model.predict(test_X)))
    print('ROC AUC score:', roc_auc_score(test_y, model.predict_proba(test_X)[::,1]))
    print('-------------------')

## Model Validation

In [ ]:
# Model validation

# cross validation
scores = cross_val_score(logreg, train_X, train_y, cv=5)
print('Cross validation scores:', scores)
print('Mean cross validation score:', scores.mean())

In [ ]:
# Bootstrap
scores = []
for i in range(1000):
    X_boot, y_boot = resample(X_train, y_train)
    logreg.fit(X_boot, y_boot)
    scores.append(logreg.score(X_test, y_test))
    
print('Mean bootstrap score:', np.mean(scores))
print('Standard deviation of bootstrap scores:', np.std(scores))

## Hyperparameter tuning

We will use random search to find the best hyperparameters for our models.

In [ ]:
# Hyperparameter tuning

# random forest hyperparameter tuning
param_grid = {'n_estimators': [10, 100, 1000],
              'max_depth': [None, 5, 10, 20],
              'max_features': ['auto', 'sqrt'],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4]}

rf = RandomForestRegressor(random_state=42)
rand_search = RandomizedSearchCV(rf, param_grid, cv=5, verbose=2, n_jobs=-1, n_iter=5)
rand_search.fit(train_X, train_y)
rand_search.best_params_, rand_search.best_score_, rand_search.best_estimator_.score(test_X, test_y)
mse = mean_squared_error(test_X, pred_y)
print(f'Mean Squared Error: {mse}')


#grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
#grid_search.best_estimator_ # best model

## Save and load the best model

In [ ]:
# save the best model

joblib.dump(rand_search.best_estimator_, 'best_model.pkl')

# load the best model
best_model = joblib.load('best_model.pkl')
best_model.score(test_X, test_y)

## Model interpretation

scikit-learn provides multiple methods for model interpretation. Here we will see feature importance and permutation importance.


In [ ]:
# Model interpretation
# feature importance

rf = RandomForestRegressor(random_state=42, n_estimators=1000, max_depth=10, max_features='sqrt', min_samples_split=5, min_samples_leaf=2)
rf.fit(train_X, train_y)
rf.feature_importances_

# plot feature importance
importances = pd.Series(rf.feature_importances_, index=selected_columns)  #este selected columns vão ser as selecionadas na feature selection, mas como não está importado não lê esta variável
importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
# feature importance based on permutation importance

perm_importance = permutation_importance(rf, test_X, test_y)
sorted_idx = perm_importance.importances_mean.argsort()
plt.barh(selected_columns[sorted_idx[:10]], perm_importance.importances_mean[sorted_idx[:10]])
plt.xlabel("Permutation Importance")
plt.show()

In [ ]:

pos_neg_idx = np.concatenate((sorted_idx[:10], sorted_idx[-10:]))
plt.barh(selected_columns[pos_neg_idx], perm_importance.importances_mean[pos_neg_idx])
plt.xlabel("Permutation Importance")
plt.show()